# Problema do Avião

O Problema de Otimização do Carregamento de um Avião de Carga, similarmente ao Problema da Garrafa, é um "problema brinquedo" de otimização utilizado como exemplo prático de aplicação de Algoritmos Genéticos.

O problema é apresentado com detalhes [aqui](http://silverio.net.br/heitor/disciplinas/ce/arquivos/aviao2020.pdf).

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from deap import base, creator, tools
from deap import algorithms

# Python complains about overwriting the Individual class. Since that is intended I have decided to hide the warning
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)

In [2]:
pop_size = 500
ngen = 10000
ind_size = 12
cxpb = 0.5
mutpb = 0.2

In [3]:
def evaluate(ind):
    obj = sum(ind[0::4]) * 0.82 + sum(ind[1::4]) * 1.15 + sum(ind[2::4]) * 0.92 + sum(ind[3::4]) * 0.75
    obj = obj / (250000 * 1.15)
    
    pen = cap_d(ind) + cap_c(ind) + cap_t(ind) + vol_d(ind) + vol_c(ind) + vol_t(ind) + carga_1(ind) + carga_2(ind) + carga_3(ind) + carga_4(ind)
    pen = pen / 3

    fitness = obj - pen
    return fitness,

In [4]:
def cap_d(ind):
    return max(0, sum(ind[:4]) - 77500) / 77500

def vol_d(ind):
    return max(0, (7.75 * ind[0] + 10.60 * ind[1] + 8.36 * ind[2] + 6.30 * ind[3]) / 1000 - 450) / 450

def cap_c(ind):
    return max(0, sum(ind[4:8]) - 115000) / 115000
   
def vol_c(ind):
    return max(0, (7.75 * ind[4] + 10.60 * ind[5] + 8.36 * ind[6] + 6.30 * ind[7]) / 1000 - 545) / 545

def cap_t(ind):
    return max(0, sum(ind[8:]) - 57500) / 57500

def vol_t(ind):
    return max(0, (7.75 * ind[8] + 10.60 * ind[9] + 8.36 * ind[10] + 6.30 * ind[11]) / 1000 - 305) / 305

def carga_1(ind):
    return max(0, sum(ind[0::4]) - 66000) / 66000

def carga_2(ind):
    return max(0, sum(ind[1::4]) - 55000) / 55000

def carga_3(ind):
    return max(0, sum(ind[2::4]) - 85000) / 85000

def carga_4(ind):
    return max(0, sum(ind[3::4]) - 40000) / 40000

In [5]:
# Create types
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [6]:
toolbox = base.Toolbox()
# Random attibute generator. Used for random initialization of individuals. Ranges from 0 to 85000
toolbox.register("attribute", np.random.randint, 85001)
# Definition of the individual. n defines the number of genes
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=ind_size)
# Definition of the population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=85000, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

df = None

In [2]:
def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, 
                                   stats=stats, halloffame=hof, verbose=False)
    
    comp_d = hof[0][:4]
    comp_c = hof[0][4:8]
    comp_t = hof[0][8:]
    comp_d.append(sum(comp_d))
    comp_c.append(sum(comp_c))
    comp_t.append(sum(comp_t))
    c1 = hof[0][0::4]
    c2 = hof[0][1::4]
    c3 = hof[0][2::4]
    c4 = hof[0][3::4]
    
    total = [sum(c1), sum(c2), sum(c3), sum(c4)]
    total.append(sum(total))
    
    df = pd.DataFrame({'Comp. Dianteiro': comp_d, 'Comp. Central': comp_c, 'Comp. Traseiro': comp_t, 'Total': total},
                      index=['Carga 1', 'Carga 2', 'Carga 3', 'Carga 4', 'Total'])
    
    display(df)
    
    print("\nCapacidade volumétrica utilizada:\n")
    print(f"Compartimento Dianteiro: {(7.75 * hof[0][0] + 10.60 * hof[0][1] + 8.36 * hof[0][2] + 6.30 * hof[0][3]) / 1000:.2f}")
    print(f"Compartimento Central: {(7.75 * hof[0][4] + 10.60 * hof[0][5] + 8.36 * hof[0][6] + 6.30 * hof[0][7]) / 1000:.2f}")
    print(f"Compartimento Traseiro: {(7.75 * hof[0][8] + 10.60 * hof[0][9] + 8.36 * hof[0][10] + 6.30 * hof[0][11]) / 1000:.2f}")
    
    print(f"\nLucro total: {0.82 * sum(c1) + 1.15 * sum(c2) + 0.92 * sum(c3) + 0.75 * sum(c4):.2f}")

In [8]:
if __name__ == "__main__":
    main()

,Comp. Dianteiro,Comp. Central,Comp. Traseiro,Total
Carga 1,2476,4997,2842,10315
Carga 2,1151,5655,19756,26562
Carga 3,33684,49162,1523,84369
Carga 4,21750,5611,9655,37016
Total,59061,65425,33776,158262



Capacidade volumétrica utilizada:

Compartimento Dianteiro: 450.01
Compartimento Central: 545.01
Compartimento Traseiro: 305.00

Lucro total: 144386.08
